In [6]:
unzip /home/ian/workspace/data/DocLayNet_core.zip -d /home/ian/workspace/data/doclaynet_extracted

NameError: name 'unzip' is not defined

In [3]:
import json
import os
from pathlib import Path
from tqdm import tqdm
import pandas as pd

ROOT = Path("/home/ian/workspace/data/doclaynet")
COCO_DIR = ROOT / "COCO"
IMG_DIR = ROOT / "PNG"
OUT_DIR = Path("/mnt/AI_NAS/datalake/source/provider=huggingface/DocLayNet/core_parquet")
OUT_DIR.mkdir(parents=True, exist_ok=True)
splits = ["train", "val", "test"]

for split in splits:
    print(f"🔄 처리 중: {split}")
    with open(COCO_DIR / f"{split}.json", "r", encoding="utf-8") as f:
        ann = json.load(f)

    images = {img["id"]: img for img in ann["images"]}
    anns = ann["annotations"]

    image_to_anns = {}
    for a in anns:
        image_to_anns.setdefault(a["image_id"], []).append(a)

    records = []
    for img_id, img in tqdm(images.items()):
        img_path = IMG_DIR / img["file_name"]
        objects = image_to_anns.get(img_id, [])

        record = {
            "image_path": str(img_path.relative_to(ROOT)),
            "image_id": img_id,
            "width": img["width"],
            "height": img["height"],
            "doc_category": img["doc_category"],
            "collection": img["collection"],
            "doc_name": img["doc_name"],
            "page_no": img["page_no"],
            "objects": objects,  
        }
        records.append(record)

    df = pd.DataFrame(records)
    out_path = OUT_DIR / f"{split}.parquet"
    df.to_parquet(out_path)
    print(f"✅ 저장 완료: {out_path} ({len(df)}개 샘플)")

🔄 처리 중: train


FileNotFoundError: [Errno 2] No such file or directory: '/home/ian/workspace/data/doclaynet/COCO/train.json'

In [1]:
from pathlib import Path

root = Path("/home/ian/workspace/data/doclaynet_extracted")
splits = ["doclaynet_train", "doclaynet_val", "doclaynet_test"]
image_exts = {".png", ".jpg", ".jpeg", ".tif", ".tiff"}

for split in splits:
    split_dir = root / split
    if not split_dir.exists():
        print(f"❌ 폴더 없음: {split_dir}")
        continue

    image_files = [f for f in split_dir.rglob("*") if f.suffix.lower() in image_exts]
    print(f"📁 {split}: {len(image_files)}장")

📁 doclaynet_train: 69375장
📁 doclaynet_val: 6489장
📁 doclaynet_test: 4999장
